In [1]:
import os

from dotenv import load_dotenv
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-academy"

load_dotenv(dotenv_path="../.env", override=True)

True

## Tracing with @traceable  
`@traceable` デコレーターは、LangSmith Python SDKからトレースをログに記録するためのシンプルな方法です。任意の関数に`@traceable`をデコレーターとして付けるだけで利用できます。

このデコレーターは、関数が呼び出されるたびにランツリー（run tree）を作成し、現在のトレース内に挿入する仕組みです。関数の入力、名前、その他の情報がLangSmithにストリームとして送信されます。関数がエラーを発生させた場合やレスポンスを返した場合、その情報もツリーに追加され、LangSmithにパッチ更新されます。そのため、エラーの原因を特定し、診断することが可能になります。これらはすべてバックグラウンドスレッドで実行されるため、アプリケーションの実行がブロックされることはありません。

In [6]:
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "openai"
MODEL_NAME = "gpt-4o-mini"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(
        metadata={"vectordb":"sklearn"}
)
def retrieve_documents(question:str):
    return retriever.invoke(question)

@traceable
def generate_answer(question:str, documents:List[str]):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {"role": "system", "content": RAG_SYSTEM_PROMPT},
        {"role": "user", "content": f"Context: {formatted_docs} \n\n Question: {question}"}
    ]
    return call_openai(messages)

@traceable(
        metadata={"model_name":MODEL_NAME,"model_provier":MODEL_PROVIDER}
)
def call_openai(messages:List[dict],
                model:str=MODEL_NAME,
                temperature:float=0.0,):
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )  

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_answer(question, documents)
    return response.choices[0].message.content




In [7]:
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)
print(ai_answer)

To trace with the @traceable decorator, simply decorate any function you want to log traces for by adding `@traceable` above the function definition. Ensure that the LANGCHAIN_TRACING_V2 environment variable is set to 'true' and that you have the necessary API key configured. This setup allows you to log traces effectively when the decorated function is called.


## Adding Metadata
LangSmithは、トレースとともに任意のメタデータを送信する機能をサポートしています。  
メタデータは、実行に付加できるキーと値のペアの集合です。メタデータを使用することで、実行を生成したアプリケーションのバージョンや、実行が生成された環境、または実行に関連付けたいその他の情報を保存することができます。タグと同様に、メタデータを使用してLangSmithのUIで実行をフィルタリングしたり、分析のために実行をグループ化することが可能です。

In [8]:
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "openai"
MODEL_NAME = "gpt-4o-mini"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="chain")
def retrieve_documents(question:str):
    return retriever.invoke(question)

@traceable(run_type="chain")
def generate_answer(question:str, documents:List[str]):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {"role": "system", "content": RAG_SYSTEM_PROMPT},
        {"role": "user", "content": f"以下の文脈をもとに質問に、日本語で答えてください。\n\n文脈: {formatted_docs} \n\n 質問: {question}"}
    ]
    return call_openai(messages)

@traceable(run_type="llm")
def call_openai(messages:List[dict],
                model:str=MODEL_NAME,
                temperature:float=0.0,):
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )  

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_answer(question, documents)
    return response.choices[0].message.content


In [9]:
question = "How do I add Metadata to a Run with @traceable?"
ai_answer = langsmith_rag(question)
print(ai_answer)

@traceableデコレーターを使用してRunにメタデータを追加するには、特別なメタデータキー（session_id、thread_id、またはconversation_idのいずれか）をトレースに添付します。具体的な手順については、関連するドキュメントを参照してください。詳細な情報は、メタデータとタグに関するページで確認できます。


In [8]:
question = "How do I add metadata at runtime?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

To add metadata at runtime, you can pass it in using the `langsmith_extra` parameter along with a unique `run_id`. For example, you can call your function like this: `rag("your question", langsmith_extra={"run_id": run_id, "metadata": {"user_id": "harrison"}})`. This allows you to log information such as User ID dynamically during execution.
